In [36]:
import pandas as pd
import numpy as np

In [37]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')


In [38]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,C27,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,B96,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,B96,S


# Feature Engineering

In [45]:
train['EngFeature'] = pd.to_numeric((train.Pclass+train.Sex*5)*900/2500)
test['EngFeature'] = pd.to_numeric((test.Pclass+train.Sex*5)*900/2500)

In [40]:
train['Family']=(train['SibSp']+train['Parch'])*2
test['Family']=(test['SibSp']+test['Parch'])*2

# Naive Bayes

In [6]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score as accuracy

In [7]:
x = np.array([train.Age,train.Family])
x = x.reshape(train.PassengerId.count(),2)
y = np.array([train.Survived])
y = y.reshape(train.PassengerId.count(),1)

In [8]:
clf = GaussianNB()

In [9]:
clf.fit(x,y)

/Users/arjuns/anaconda/envs/py3/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None)

In [10]:
testx = np.array([test.Age,test.Family])
testx = testx.reshape(test.PassengerId.count(),2)
testy = np.array([test.Survived])
testy = testy.reshape(test.PassengerId.count(),1)

In [11]:
pred = clf.predict(testx)

In [12]:
accuracy(pred,testy)

0.66000000000000003

# Decision Tree

In [67]:
from sklearn import tree
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,EngFeature,Family
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,C27,S,1.08,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C,2.16,2
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,B96,S,2.88,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S,2.16,2
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,B96,S,1.08,0


In [135]:
x = np.array([train.Sex,train.SibSp])
x = x.reshape(train.PassengerId.count(),2)
y = np.array([train.Survived])
y = y.reshape(train.PassengerId.count(),1)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x, y)
testx = np.array([test.Sex,test.SibSp])
testx = testx.reshape(test.PassengerId.count(),2)
testy = np.array([test.Survived])
testy = testy.reshape(test.PassengerId.count(),1)
pred = clf.predict(testx)
accuracy(pred,testy)

0.66500000000000004

# SVM

In [98]:
from sklearn.svm import SVC

In [124]:
x = np.array([train.EngFeature,train.Family])
x = x.reshape(train.PassengerId.count(),2)
y = np.array([train.Survived])
y = y.reshape(train.PassengerId.count(),1)
testx = np.array([test.EngFeature,test.Family])
testx = testx.reshape(test.PassengerId.count(),2)
testy = np.array([test.Survived])
testy = testy.reshape(test.PassengerId.count(),1)
clf = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=5, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
clf.fit(x,y)
pred = clf.predict(testx)
accuracy(pred,testy)

/Users/arjuns/anaconda/envs/py3/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.65500000000000003

# Submisson

In [136]:
trueTest = pd.read_csv('data/actual_test.csv')
#Fill NA
trueTest.Age.fillna(value=round(trueTest.Age.mean()),inplace=True)
for x in range(trueTest.Cabin.size):
    rep = np.random.choice(['B57', 'B59', 'B63', 'B66'])
    if pd.isnull(trueTest.Cabin[x]):
        trueTest.set_value(x,'Cabin',rep)
trueTest.Embarked.fillna(value=str(trueTest.Embarked.mode()),inplace=True)
trueTest.Fare.fillna(value=float(trueTest.Fare.mean()),inplace=True)
for x in range(trueTest.PassengerId.size):
    if (trueTest.Sex[x]=='female'):
        trueTest.set_value(x,'Sex',int(1))
    elif (trueTest.Sex[x]=='male'):
        trueTest.set_value(x,'Sex',int(0))
trueTest['EngFeature'] = pd.to_numeric((trueTest.Pclass+trueTest.Sex*5)*900/2500)
trueTest['Family']=(trueTest['SibSp']+trueTest['Parch'])*2
trx = np.array([trueTest.Sex,trueTest.SibSp])
trx = trx.reshape(trueTest.PassengerId.count(),2)
truePredictions = clf.predict(trx)
truePrediction = truePredictions.reshape(trueTest.PassengerId.count(),1)
passs = np.array([trueTest.PassengerId])
passs = passs.reshape(trueTest.PassengerId.count(),1)
newfile = open(input()+'.txt','w')
newfile.writelines('PassengerId,Survived\n')
for x in range(len(truePrediction)):
    s = "{},{}\n".format(passs[x][0],truePrediction[x][0])
    newfile.writelines(s)
newfile.close()

DecisionTree-NoFeatEng-Sex-Sibsp
